In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib as plt
import numpy as np
import sklearn
import matplotlib.pyplot as mtplt
import re
from collections import Counter
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
# for pre - processing
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup, Embedding
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train = pd.read_csv('featureEngineered3.csv')

In [ ]:
train.head()

,Unnamed: 0,tweet,tweettype,charCount,wordCount,sentenceCount,capCharCount,capWordCount,quotedWordCount,stopwordCount,uniqueWordCount,hashCount,mentionCount,avgWordLen,modTweet,! count,""" count",# count,$ count,% count,& count,' count,( count,) count,* count,+ count,", count",- count,. count,/ count,: count,; count,< count,= count,> count,? count,@ count,[ count,\ count,] count,^ count,_ count,` count,{ count,| count,} count,~ count,lemmatizedText
0,0,@ZubairSabirPTI pls dont insult the word 'Molna',anger,49,7,1,6,0,0,1,7,0,0,7.000000,pls dont insult the word 'molna',0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pls dont insult word
1,1,@ArcticFantasy I would have almost took offens...,anger,81,14,1,4,2,0,5,13,0,0,5.785714,would have almost took offense to this if actu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,would almost take offense actually snap
2,2,@IllinoisLoyalty that Rutgers game was an abom...,anger,114,20,3,6,0,0,8,20,0,0,5.700000,that rutgers game was an abomination an affron...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rutgers game abomination affront god man must ...
3,3,@CozanGaming that's what lisa asked before she...,anger,90,16,2,3,1,0,7,16,0,0,5.625000,that's what lisa asked before she started ragi...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,lisa ask start rag call heh
4,4,Sometimes I get mad over something so minuscul...,anger,133,25,1,3,2,0,7,22,0,0,5.320000,sometimes get mad over something so minuscule ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sometimes get mad something minuscule try ruin...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['tweet'])

In [ ]:
print(bow)

  (0, 224)	1
  (0, 960)	1
  (1, 863)	1
  (1, 17)	1
  (2, 326)	1
  (2, 338)	1
  (2, 539)	1
  (3, 48)	1
  (3, 791)	1
  (4, 533)	1
  (4, 876)	1
  (4, 488)	1
  (4, 490)	2
  (4, 514)	1
  (4, 449)	1
  (5, 533)	1
  (5, 876)	1
  (5, 488)	1
  (5, 490)	2
  (5, 514)	1
  (5, 449)	1
  (5, 36)	1
  (6, 17)	1
  (6, 490)	1
  (6, 840)	1
  :	:
  (43953, 364)	1
  (43953, 984)	1
  (43953, 222)	1
  (43953, 129)	1
  (43954, 309)	1
  (43956, 518)	1
  (43956, 198)	1
  (43956, 377)	1
  (43956, 580)	1
  (43957, 539)	1
  (43957, 198)	2
  (43957, 377)	1
  (43957, 952)	1
  (43957, 508)	1
  (43957, 579)	1
  (43958, 401)	1
  (43958, 597)	1
  (43958, 307)	1
  (43958, 72)	1
  (43958, 904)	1
  (43958, 161)	1
  (43958, 589)	1
  (43958, 977)	1
  (43959, 871)	1
  (43959, 847)	1


In [ ]:
train['tweettype'].value_counts()

sadness       14484
neutral        9644
joy            6111
love           3842
enthusiasm     2535
anger          2374
surprise       2187
relief         1526
fear           1257
Name: tweettype, dtype: int64

In [ ]:
labels = np.array(train['tweettype'])
y = []
for i in range(len(labels)):
    if labels[i] == 'sadness':
        y.append(0)
    elif labels[i] == 'neutral':
        y.append(1)
    elif labels[i] == 'joy':
        y.append(2)
    elif labels[i] == 'love':
        y.append(3)
    elif labels[i] == 'enthusiasm':
        y.append(4)
    elif labels[i] == 'anger':
        y.append(5)
    elif labels[i] == 'surprise':
        y.append(6)
    elif labels[i] == 'relief':
        y.append(7)
    elif labels[i] == 'fear':
        y.append(8)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 9, dtype="float32")
del y

In [ ]:
def count_punctuations(tweet):
    punctuations='!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~'
    d=dict()
    for i in punctuations:
        d[str(i)+' count']=tweet.count(i)
    return d

for i, tweet in enumerate(train['tweet']):
  dictPunct = count_punctuations(tweet)
  for key,val in dictPunct.items():
    train.loc[i,key] = val

In [ ]:
train.head()

,Unnamed: 0,index,tweet,tweettype,charCount,wordCount,sentenceCount,capCharCount,capWordCount,quotedWordCount,stopwordCount,uniqueWordCount,hashCount,mentionCount,punctCount,avgWordLen,modTweet,! count,""" count",# count,$ count,% count,& count,' count,( count,) count,* count,+ count,", count",- count,. count,/ count,: count,; count,< count,= count,> count,? count,@ count,[ count,\ count,] count,^ count,_ count,` count,{ count,| count,} count,~ count
0,0,0,@ZubairSabirPTI pls dont insult the word 'Molna',anger,49,7,1,6,0,0,1,7,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",7.000000,"['ZubairSabirPTI', 'pls', 'dont', 'insult', 'w...",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,@ArcticFantasy I would have almost took offens...,anger,81,14,1,4,2,0,5,13,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.785714,"['ArcticFantasy', 'I', 'would', 'almost', 'tak...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,@IllinoisLoyalty that Rutgers game was an abom...,anger,114,20,3,6,0,0,8,20,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.700000,"['IllinoisLoyalty', 'Rutgers', 'game', 'abomin...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,@CozanGaming that's what lisa asked before she...,anger,90,16,2,3,1,0,7,16,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.625000,"['CozanGaming', 'lisa', 'ask', 'start', 'rag',...",0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,Sometimes I get mad over something so minuscul...,anger,133,25,1,3,2,0,7,22,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.320000,"['Sometimes', 'I', 'get', 'mad', 'something', ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.to_csv('featureEngineered.csv')

In [ ]:
train.describe()

,Unnamed: 0,index,charCount,wordCount,sentenceCount,capCharCount,capWordCount,quotedWordCount,stopwordCount,uniqueWordCount,hashCount,mentionCount,avgWordLen,! count,""" count",# count,$ count,% count,& count,' count,( count,) count,* count,+ count,", count",- count,. count,/ count,: count,; count,< count,= count,> count,? count,@ count,[ count,\ count,] count,^ count,_ count,` count,{ count,| count,} count,~ count
count,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.0,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.0,43960.000000,43960.0,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000,43960.000000
mean,21979.500000,21979.500000,75.395928,13.473703,1.693881,3.355232,0.605255,0.000409,4.802320,12.878890,0.003594,0.005300,5.800445,0.523658,0.0,0.119722,0.005232,0.002298,0.088330,0.431893,0.026729,0.031847,0.024090,0.004663,0.312011,0.099568,1.257370,0.153753,0.086283,0.100978,0.0,0.007143,0.0,0.164081,0.518358,0.001774,0.011146,0.002866,0.001934,0.063171,0.001388,0.000409,0.001388,0.000409,0.006392
std,12690.303254,12690.303254,37.174199,6.988966,0.939460,5.094857,1.277425,0.020231,3.416613,6.420676,0.063889,0.072923,1.328967,1.305959,0.0,0.658868,0.095961,0.053701,0.408068,0.724029,0.196166,0.199946,0.248908,0.079530,0.649891,0.412481,1.874228,0.670621,0.334879,0.431920,0.0,0.103814,0.0,1.172758,0.607572,0.042623,0.189618,0.054723,0.069255,0.320219,0.060878,0.023362,0.047913,0.023362,0.106137
min,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10989.750000,10989.750000,44.000000,8.000000,1.000000,1.000000,0.000000,0.000000,2.000000,8.000000,0.000000,0.000000,5.040000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,21979.500000,21979.500000,71.000000,13.000000,1.000000,2.000000,0.000000,0.000000,4.000000,12.000000,0.000000,0.000000,5.555556,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,32969.250000,32969.250000,107.000000,19.000000,2.000000,4.000000,1.000000,0.000000,7.000000,18.000000,0.000000,0.000000,6.222222,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,43959.000000,43959.000000,286.000000,34.000000,21.000000,131.000000,28.000000,1.000000,20.000000,30.000000,2.000000,2.000000,63.000000,76.000000,0.0,14.000000,5.000000,3.000000,8.000000,7.000000,18.000000,6.000000,14.000000,4.000000,9.000000,14.000000,73.000000,13.000000,13.000000,8.000000,0.0,10.000000,0.0,98.000000,10.000000,2.000000,10.000000,2.000000,8.000000,22.000000,9.000000,3.000000,4.000000,3.000000,9.000000


max words is observed as 34

In [ ]:
train = train.drop('punctCount', axis = 1)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43960 entries, 0 to 43959
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       43960 non-null  int64  
 1   tweet            43960 non-null  object 
 2   tweettype        43960 non-null  object 
 3   charCount        43960 non-null  int64  
 4   wordCount        43960 non-null  int64  
 5   sentenceCount    43960 non-null  int64  
 6   capCharCount     43960 non-null  int64  
 7   capWordCount     43960 non-null  int64  
 8   quotedWordCount  43960 non-null  int64  
 9   stopwordCount    43960 non-null  int64  
 10  uniqueWordCount  43960 non-null  int64  
 11  hashCount        43960 non-null  int64  
 12  mentionCount     43960 non-null  int64  
 13  avgWordLen       43960 non-null  float64
 14  modTweet         43874 non-null  object 
 15  ! count          43960 non-null  float64
 16  " count          43960 non-null  float64
 17  # count     

In [ ]:
train = train.drop(['Unnamed: 0','index'], axis = 1)

In [ ]:
def featureEngineering(tweet):
    # Lower case tweet
    tweetMod = tweet.lower()
    # Replace URLs with a space in the message
    tweetMod = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweetMod)
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    tweetMod = re.sub('\$[a-zA-Z0-9]*', ' ', tweetMod)
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    tweetMod = re.sub('\@[a-zA-Z0-9]*', ' ', tweetMod)
    # Replace everything not a letter or apostrophe with a space
    tweetMod = re.sub('[^a-zA-Z\']', ' ', tweetMod)
    # Remove single letter words
    tweetMod = ' '.join( [w for w in tweetMod.split() if len(w)>1] )
    
    return tweetMod

# Process for all tweets
train['modTweet'] = [featureEngineering(tweet) for tweet in train['tweet']]

In [ ]:
def tokenizeTweet(tweet, option):
  '''
  Tokenize the tweet, different methods - change as per accuracy and score requirements
    1: Use python split() function
    2: Use regex to extract alphabets plus 's and 't
    3: Use NLTK word_tokenize()
    4: Use NLTK word_tokenize(), remove stop words and apply lemmatization
  '''
  if option == 1:
    return tweet.split()
  elif option == 2:
    return re.findall(r'\b([a-zA-Z]+n\'t|[a-zA-Z]+\'s|[a-zA-Z]+)\b', tweet)
  elif option == 3:
    return [word for word in word_tokenize(tweet) if (word.isalpha()==1)]
  elif option == 4:
    words = [word for word in word_tokenize(tweet) if (word.isalpha()==1)]
    # Remove stop words
    stop = set(stopwords.words('english'))
    words = [word for word in words if (word not in stop)]
    # Lemmatize words (first noun, then verb)
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized = [wnl.lemmatize(wnl.lemmatize(word, 'n'), 'v') for word in words]
    return lemmatized
  else:
    logger.warn("Use a number between 1 and 4")
    return []

wordsSet = set()
for tweet in train['modTweet']:
  listOfWords = tokenizeTweet(tweet, 4)
  wordsSet.update(listOfWords)

In [ ]:
print(wordsSet)

{'amr', 'bald', 'secondstar', 'tivo', 'newww', 'synth', 'photoshoots', 'awe', 'careful', 'nest', 'pancake', 'diff', 'grease', 'alphonso', 'funtime', 'mothersday', 'fricken', 'songwriting', 'twilightguy', 'bronchitis', 'haaaayley', 'inconclusive', 'whuahahhaha', 'blackmen', 'energy', 'catholic', 'getcha', 'breadtalk', 'speedway', 'hamburg', 'portland', 'trample', 'pierre', 'crispy', 'adam', 'disband', 'electronics', 'frustration', 'dentention', 'tamron', 'ehhehehe', 'ectopic', 'clone', 'jenni', 'every', 'nutty', 'nje', 'schoool', 'knack', 'bubblegum', 'pavement', 'clinto', 'tiz', 'yoghurt', 'spring', 'jeepjeep', 'compact', 'reese', 'specially', 'sentosa', 'heater', 'osler', 'bloom', 'drs', 'dota', 'succeed', 'purdy', 'lso', 'purely', 'harrassment', 'desication', 'gl', 'thoracic', 'rembrandt', 'shingle', 'nawazsharif', 'softsynths', 'self', 'torus', 'sherraton', 'lucario', 'sue', 'aboard', 'yett', 'tps', 'mpls', 'noida', 'bootay', 'babsi', 'attractive', 'avoid', 'benzo', 'identify', 'twe

In [ ]:
print(len(wordsSet))

27608


In [ ]:
def lemmatizeTweet(tweet):
  words = [word for word in word_tokenize(tweet) if (word.isalpha()==1)]
  # Remove stop words
  stop = set(stopwords.words('english'))
  words = [word for word in words if (word not in stop)]
  # Lemmatize words (first noun, then verb)
  wnl = nltk.stem.WordNetLemmatizer()
  lemmatized = [wnl.lemmatize(wnl.lemmatize(word, 'n'), 'v') for word in words]
  return " ".join(lemmatized)

train['lemmatizedText'] = train["modTweet"].apply(lambda x:lemmatizeTweet(x))

In [ ]:
train.head()

,Unnamed: 0,tweet,tweettype,charCount,wordCount,sentenceCount,capCharCount,capWordCount,quotedWordCount,stopwordCount,uniqueWordCount,hashCount,mentionCount,avgWordLen,modTweet,! count,""" count",# count,$ count,% count,& count,' count,( count,) count,* count,+ count,", count",- count,. count,/ count,: count,; count,< count,= count,> count,? count,@ count,[ count,\ count,] count,^ count,_ count,` count,{ count,| count,} count,~ count,lemmatizedText
0,0,@ZubairSabirPTI pls dont insult the word 'Molna',anger,49,7,1,6,0,0,1,7,0,0,7.000000,pls dont insult the word 'molna',0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pls dont insult word
1,1,@ArcticFantasy I would have almost took offens...,anger,81,14,1,4,2,0,5,13,0,0,5.785714,would have almost took offense to this if actu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,would almost take offense actually snap
2,2,@IllinoisLoyalty that Rutgers game was an abom...,anger,114,20,3,6,0,0,8,20,0,0,5.700000,that rutgers game was an abomination an affron...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rutgers game abomination affront god man must ...
3,3,@CozanGaming that's what lisa asked before she...,anger,90,16,2,3,1,0,7,16,0,0,5.625000,that's what lisa asked before she started ragi...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,lisa ask start rag call heh
4,4,Sometimes I get mad over something so minuscul...,anger,133,25,1,3,2,0,7,22,0,0,5.320000,sometimes get mad over something so minuscule ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sometimes get mad something minuscule try ruin...


In [ ]:
# Convert the dataframe into a tensorflow dataset
def dataframe_to_dataset(dataframe, labels):
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

# Modify the categorical encoding function to incorporate the embedding layer
def encode_categorical_feature_embedding(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    # Modify the lookup class to output integers instead of one - hot encoding
    lookup = lookup_class() 

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    length = len(wordsSet)
    # Add the embedding layer with output_dim equal to floor(num_categories/2)
    encoded_feature = layers.Embedding(length + 1, length // 2, input_length = 1)(encoded_feature)
    # To make it compatible to a dense layer
    encoded_feature1 = layers.Flatten()(encoded_feature)
    return encoded_feature1


In [ ]:
tweetTrain = train.drop('tweettype', axis = 1)

In [ ]:
tweetTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43960 entries, 0 to 43959
Data columns (total 47 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       43960 non-null  int64  
 1   tweet            43960 non-null  object 
 2   charCount        43960 non-null  int64  
 3   wordCount        43960 non-null  int64  
 4   sentenceCount    43960 non-null  int64  
 5   capCharCount     43960 non-null  int64  
 6   capWordCount     43960 non-null  int64  
 7   quotedWordCount  43960 non-null  int64  
 8   stopwordCount    43960 non-null  int64  
 9   uniqueWordCount  43960 non-null  int64  
 10  hashCount        43960 non-null  int64  
 11  mentionCount     43960 non-null  int64  
 12  avgWordLen       43960 non-null  float64
 13  modTweet         43874 non-null  object 
 14  ! count          43960 non-null  float64
 15  " count          43960 non-null  float64
 16  # count          43960 non-null  float64
 17  $ count     

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweetTrain,labels,train_size = 0.7)
print (len(X_train),len(X_test),len(y_train),len(y_test))

30771 13189 30771 13189


In [ ]:
# Convert to dataset
train_ds = dataframe_to_dataset(X_train, y_train)
val_ds = dataframe_to_dataset(X_test, y_test)

# Create batches of size 128
train_ds = train_ds.batch(128)
val_ds = val_ds.batch(128)

In [ ]:
numericalFeatures = X_train.drop(['lemmatizedText', 'tweet', 'modTweet'], axis = 1)

In [ ]:
# Encoding the object datatypes
lemmatizedText = keras.Input(shape=(1,), name="lemmatizedText", dtype="string")

all_inputs = [lemmatizedText]
for column in numericalFeatures:
  all_inputs.append(keras.Input(shape=(1,), name=column))

In [ ]:
# Normalize the numerical inputs
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature

In [ ]:
lemmatizedText_encoded = encode_categorical_feature_embedding(lemmatizedText, "lemmatizedText", train_ds, True)

all_features_list = [lemmatizedText_encoded]
for i, column in enumerate(numericalFeatures):
  all_features_list.append(encode_numerical_feature(all_inputs[i], column, train_ds))
all_features = layers.concatenate(all_features_list)

x = layers.Dense(10, activation='relu')(all_features)
output = layers.Dense(1, activation = 'linear')(x)
model = keras.Model(all_inputs, output)

In [ ]:
train.to_csv('featureEngineered3.csv')

In [ ]:
# Visualizing the constructed model
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
print(train['lemmatizedText'])

0                                     pls dont insult word
1                  would almost take offense actually snap
2        rutgers game abomination affront god man must ...
3                              lisa ask start rag call heh
4        sometimes get mad something minuscule try ruin...
                               ...                        
43955                                                     
43956                                happy mother day love
43957    happy mother day mommy woman man long someone day
43958    wassup beautiful follow peep new hit single ww...
43959    bullet train tokyo gf visit japan since thursd...
Name: lemmatizedText, Length: 43960, dtype: object


In [ ]:
wordsSet = set()
for tweet in train['lemmatizedText']:
  listWords = list(tweet)
  wordsSet.update(listWords)

In [ ]:
listWords = list(wordsSet)

In [ ]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in listWords:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'amr': 1, 'bald': 2, 'secondstar': 3, 'tivo': 4, 'newww': 5, 'synth': 6, 'photoshoots': 7, 'awe': 8, 'careful': 9, 'nest': 10, 'pancake': 11, 'diff': 12, 'grease': 13, 'alphonso': 14, 'funtime': 15, 'mothersday': 16, 'fricken': 17, 'songwriting': 18, 'twilightguy': 19, 'bronchitis': 20, 'haaaayley': 21, 'inconclusive': 22, 'whuahahhaha': 23, 'blackmen': 24, 'energy': 25, 'catholic': 26, 'getcha': 27, 'breadtalk': 28, 'speedway': 29, 'hamburg': 30, 'portland': 31, 'trample': 32, 'pierre': 33, 'crispy': 34, 'adam': 35, 'disband': 36, 'electronics': 37, 'frustration': 38, 'dentention': 39, 'tamron': 40, 'ehhehehe': 41, 'ectopic': 42, 'clone': 43, 'jenni': 44, 'every': 45, 'nutty': 46, 'nje': 47, 'schoool': 48, 'knack': 49, 'bubblegum': 50, 'pavement': 51, 'clinto': 52, 'tiz': 53, 'yoghurt': 54, 'spring': 55, 'jeepjeep': 56, 'compact': 57, 'reese': 58, 'specially': 59, 'sentosa': 60, 'heater': 61, 'osler': 62, 'bloom': 63, 'drs': 64, 'dota': 65, 'succeed': 66, 'purdy': 67, 'ls

In [ ]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'amr', 2: 'bald', 3: 'secondstar', 4: 'tivo', 5: 'newww', 6: 'synth', 7: 'photoshoots', 8: 'awe', 9: 'careful', 10: 'nest', 11: 'pancake', 12: 'diff', 13: 'grease', 14: 'alphonso', 15: 'funtime', 16: 'mothersday', 17: 'fricken', 18: 'songwriting', 19: 'twilightguy', 20: 'bronchitis', 21: 'haaaayley', 22: 'inconclusive', 23: 'whuahahhaha', 24: 'blackmen', 25: 'energy', 26: 'catholic', 27: 'getcha', 28: 'breadtalk', 29: 'speedway', 30: 'hamburg', 31: 'portland', 32: 'trample', 33: 'pierre', 34: 'crispy', 35: 'adam', 36: 'disband', 37: 'electronics', 38: 'frustration', 39: 'dentention', 40: 'tamron', 41: 'ehhehehe', 42: 'ectopic', 43: 'clone', 44: 'jenni', 45: 'every', 46: 'nutty', 47: 'nje', 48: 'schoool', 49: 'knack', 50: 'bubblegum', 51: 'pavement', 52: 'clinto', 53: 'tiz', 54: 'yoghurt', 55: 'spring', 56: 'jeepjeep', 57: 'compact', 58: 'reese', 59: 'specially', 60: 'sentosa', 61: 'heater', 62: 'osler', 63: 'bloom', 64: 'drs', 65: 'dota', 66: 'succeed', 67: 'purdy', 68:

In [ ]:
example_sequence = [vocab[word] for word in ["tivo", "rmbr", "shivshankar"]]
print(example_sequence)

[4, 13881, 19975]


In [ ]:
train['lemmatizedText'].values

array(['pls dont insult word', 'would almost take offense actually snap',
       'rutgers game abomination affront god man must never speak', ...,
       'happy mother day mommy woman man long someone day',
       'wassup beautiful follow peep new hit single www myspace com ipsohot def wat video',
       'bullet train tokyo gf visit japan since thursday vacation sightsee gaijin godzill'],
      dtype=object)

In [ ]:
tokenizer = Tokenizer(num_words = len(wordsSet), split = ' ')

In [ ]:
tokenizer.fit_on_texts(train['lemmatizedText'].values)

In [ ]:
X = tokenizer.texts_to_sequences(train['lemmatizedText'])
X = pad_sequences(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,labels,train_size = 0.7)
print (len(X_train),len(X_test),len(y_train),len(y_test))

30771 13189 30771 13189


In [ ]:
embed_dim = 128
lstm_out = 196
vocabSize = len(wordsSet)
model = Sequential()
model.add(Embedding(vocabSize, embed_dim, input_length = 28))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(9, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train,validation_data = (X_test,y_test),epochs = 10, batch_size=32)

Epoch 1/10
962/962 [==============================] - 188s 192ms/step - loss: 1.6829 - accuracy: 0.4113 - val_loss: 1.5254 - val_accuracy: 0.4673
Epoch 2/10
962/962 [==============================] - 186s 193ms/step - loss: 1.3467 - accuracy: 0.5387 - val_loss: 1.5380 - val_accuracy: 0.4581
Epoch 3/10
962/962 [==============================] - 185s 192ms/step - loss: 1.0712 - accuracy: 0.6379 - val_loss: 1.6968 - val_accuracy: 0.4386
Epoch 4/10
962/962 [==============================] - 184s 191ms/step - loss: 0.8259 - accuracy: 0.7226 - val_loss: 1.8434 - val_accuracy: 0.4219
Epoch 5/10
962/962 [==============================] - 184s 191ms/step - loss: 0.6567 - accuracy: 0.7754 - val_loss: 2.0926 - val_accuracy: 0.4103
Epoch 6/10
962/962 [==============================] - 183s 191ms/step - loss: 0.5540 - accuracy: 0.8084 - val_loss: 2.3355 - val_accuracy: 0.4040
Epoch 7/10
962/962 [==============================] - 182s 189ms/step - loss: 0.4834 - accuracy: 0.8323 - val_loss: 2.6188 -

In [ ]:
# plotting the accuracies for the training epochs
plt.figure(1)
plt.plot(model['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.savefig('1b_accuracies.png')

TypeError: ignored